<a href="https://colab.research.google.com/github/devroopsaha744/HateSpeechDetect-text/blob/main/Hatespeech_detection_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets -q
!pip install nltk -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.4 MB/s eta 0:00:00


In [32]:
!pip install gensim -q

In [33]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [60]:
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

In [35]:
from datasets import load_dataset
data = load_dataset("tdavidson/hate_speech_offensive")

In [36]:
data_split = data['train'].train_test_split(test_size = 0.3)

In [38]:
train = data_split['train']
test = data_split['test']

In [39]:
train

Dataset({
    features: ['count', 'hate_speech_count', 'offensive_language_count', 'neither_count', 'class', 'tweet'],
    num_rows: 17348
})

In [40]:
test

Dataset({
    features: ['count', 'hate_speech_count', 'offensive_language_count', 'neither_count', 'class', 'tweet'],
    num_rows: 7435
})

In [41]:
train_df =  train.to_pandas()
test_df = test.to_pandas()

In [42]:
train_df

,count,hate_speech_count,offensive_language_count,neither_count,class,tweet
0,6,1,1,4,2,"RT @sorryimalex: ""now 5sos is starting to twee..."
1,3,0,3,0,1,"RT @nandotrash: Tell a girl to ""calm down"" she..."
2,3,0,3,0,1,"RT @thecoreyholcomb: Twitter family, I promise..."
3,6,0,1,5,2,@Notsosweetpea SOO when we hanging out and mak...
4,3,0,3,0,1,"So, #brony-folk far and wide are throwing bitc..."
...,...,...,...,...,...,...
17343,3,0,3,0,1,Nobody wit me im on all alone shit so dont thi...
17344,3,1,2,0,1,@GYPMATIC45 i pissed alotta ppl off on fb i wa...
17345,3,0,3,0,1,Miss ma niggah leo
17346,3,0,3,0,1,"If eating the pussy makes you happy, I'm all f..."


In [43]:
test_df

,count,hate_speech_count,offensive_language_count,neither_count,class,tweet
0,3,2,1,0,0,@deno_5 @kaitlyninst0ne you're reading you coon
1,3,0,3,0,1,Slide in your bitch Dms like &#128166; &#12794...
2,3,0,3,0,1,&#8220;@Jaime_915: @Thompson_Era @HannahAlisee...
3,3,0,0,3,2,RT @TheEagleFazi: Almost 83% of children in Ba...
4,3,1,2,0,1,Well your a little cunt !
...,...,...,...,...,...,...
7430,3,0,2,1,1,RT @haztv: Stop forcing ya kids to take these ...
7431,3,2,1,0,0,"RT @ItsSonGoku: When niggas ask me ""Where the ..."
7432,3,3,0,0,0,https://t.co/jtLZErgxI0 Leo get the fuck off o...
7433,3,0,3,0,1,"I might freak the bitch, but dont need the bit..."


In [44]:
# Load the stop words
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    # Remove special characters and punctuation
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stop words and lemmatize the tokens
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    # Join the tokens back into a single string
    text = ' '.join(tokens)
    return text

# Apply the preprocessing function to the 'text' column
train_df['tweet'] = train_df['tweet'].apply(preprocess_text)

In [48]:
test_df['tweet'] = test_df['tweet'].apply(preprocess_text)

In [45]:
train_df

,count,hate_speech_count,offensive_language_count,neither_count,class,tweet
0,6,1,1,4,2,rt sorryimalex 5sos starting tweet hashtag the...
1,3,0,3,0,1,rt nandotrash tell girl calm shell act like ki...
2,3,0,3,0,1,rt thecoreyholcomb twitter family promise put ...
3,6,0,1,5,2,notsosweetpea soo hanging makin brownie
4,3,0,3,0,1,bronyfolk far wide throwing bitch fit equestri...
...,...,...,...,...,...,...
17343,3,0,3,0,1,nobody wit im alone shit dont think im actin f...
17344,3,1,2,0,1,gypmatic45 pissed alotta ppl fb flooding newsf...
17345,3,0,3,0,1,miss niggah leo
17346,3,0,3,0,1,eating pussy make happy im


In [46]:
train_df['tokens'] = train_df['tweet'].apply(word_tokenize)

# Train Word2Vec model
model = Word2Vec(sentences=train_df['tokens'], vector_size=100, window=5, min_count=1, workers=4)

# Save the model for future use
model.save("word2vec.model")

# Load the model
# model = Word2Vec.load("word2vec.model")


In [51]:
test_df['tokens'] = test_df['tweet'].apply(word_tokenize)

In [52]:
def get_average_word2vec(tokens, model, vector_size):
    # Initialize an empty vector
    vec = np.zeros(vector_size)
    count = 0
    for word in tokens:
        if word in model.wv:
            vec += model.wv[word]
            count += 1
    if count != 0:
        vec /= count
    return vec

vector_size = model.vector_size

train_df['text_vec'] = train_df['tokens'].apply(lambda x: get_average_word2vec(x, model, vector_size))
test_df['text_vec'] = test_df['tokens'].apply(lambda x: get_average_word2vec(x, model, vector_size))

# Convert lists of vectors to a 2D array for the classifier
X_train = np.stack(train_df['text_vec'].values)
y_train = train_df['class'].values
X_test = np.stack(test_df['text_vec'].values)
y_test = test_df['class'].values


In [62]:
clf1 = SVC()
clf2 = LogisticRegression()
clf3 = DecisionTreeClassifier()
clf4 = RandomForestClassifier()
clf5 = GradientBoostingClassifier()
clf6 = XGBClassifier()
clf7 = AdaBoostClassifier()

classifier = [clf1, clf2, clf3, clf4, clf5, clf6, clf7]
for clf in classifier:
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(f"The Classification report of {clf} is: \n {classification_report(y_test, y_pred)}")
  print("\n")

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


The Classification report of SVC() is: 
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       438
           1       0.78      0.99      0.87      5681
           2       0.63      0.12      0.20      1316

    accuracy                           0.78      7435
   macro avg       0.47      0.37      0.36      7435
weighted avg       0.71      0.78      0.70      7435





/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels wit

The Classification report of LogisticRegression() is: 
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       438
           1       0.84      0.97      0.90      5681
           2       0.71      0.49      0.58      1316

    accuracy                           0.83      7435
   macro avg       0.52      0.49      0.49      7435
weighted avg       0.77      0.83      0.79      7435



The Classification report of DecisionTreeClassifier() is: 
               precision    recall  f1-score   support

           0       0.12      0.13      0.12       438
           1       0.87      0.88      0.87      5681
           2       0.58      0.53      0.55      1316

    accuracy                           0.77      7435
   macro avg       0.52      0.51      0.52      7435
weighted avg       0.77      0.77      0.77      7435



The Classification report of RandomForestClassifier() is: 
               precision    recall  f1-score   support

   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
